In [2]:
# LLaMA 3.1 1B Medical SOAP Fine-tuning with LoRA
# Google Colab Notebook for fine-tuning on MTS_Dialogue-Clinical_Note

# =============================================================================
# STEP 1: Setup and Installation
# =============================================================================

!pip install -q transformers datasets peft accelerate bitsandbytes trl
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q huggingface_hub

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from huggingface_hub import login

# Check GPU
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")


# =============================================================================
# STEP 2: Login to Hugging Face
# =============================================================================

login()  # Interactive login, paste your token when prompted

# =============================================================================
# STEP 3: Model Configuration
# =============================================================================

model_name = "andrijdavid/Llama3-1B-Base"
dataset_name = "har1/MTS_Dialogue-Clinical_Note"

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)


# =============================================================================
# STEP 4: Load Dataset and Tokenizer
# =============================================================================

print("Loading dataset...")
dataset = load_dataset(dataset_name)
print(f"Dataset loaded. Train size: {len(dataset['train'])}")

# Split train dataset into 90% train / 10% validation
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
print(f"Train size: {len(dataset['train'])}, Validation size: {len(dataset['test'])}")

print("\nDataset structure:")
print("Columns:", dataset['train'].column_names)
print("\nFirst example:")
print(dataset['train'][0])

print("\nLoading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


# =============================================================================
# STEP 5: Data Preprocessing
# =============================================================================

MAX_LENGTH = 1024

def format_medical_dialogue(example):
    dialogue_text = example.get("dialogue", "")
    soap_text = "Please generate a SOAP summary for this dialogue."

    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a medical assistant. Given a medical dialogue between a patient and healthcare provider, generate a SOAP (Subjective, Objective, Assessment, Plan) summary.
<|eot_id|><|start_header_id|>user<|end_header_id|>
Medical Dialogue: {dialogue_text}
Please provide a SOAP summary for this medical dialogue.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{soap_text}<|eot_id|>"""

    return {"text": prompt[:MAX_LENGTH]}  # truncate if too long

formatted_dataset = dataset.map(format_medical_dialogue)
print("Sample formatted example:", formatted_dataset['train'][0]['text'][:500] + "...")


# =============================================================================
# STEP 6: Load Model
# =============================================================================

torch.cuda.empty_cache()
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)
torch.cuda.empty_cache()

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
torch.cuda.empty_cache()


# =============================================================================
# STEP 7: Training Configuration
# =============================================================================

training_args = TrainingArguments(
    output_dir="./llama1b-medical-soap-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    logging_steps=10,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    warmup_steps=100,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    gradient_checkpointing=True,
    dataloader_num_workers=0,
    max_steps=-1,
    report_to="none",
    weight_decay=0.01,
    seed=3407,
    save_total_limit=2,
    push_to_hub=False,
)


# =============================================================================
# STEP 8: Initialize Trainer
# =============================================================================

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset['train'],
    eval_dataset=formatted_dataset['test'],  # validation dataset provided
    args=training_args,
    formatting_func=lambda x: x["text"],
)
print("✅ SFTTrainer initialized successfully")


# =============================================================================
# STEP 9: Training
# =============================================================================

print("Starting training...")
trainer.train()

print("Saving model...")
trainer.save_model()
tokenizer.save_pretrained("./llama1b-medical-soap-lora")


# =============================================================================
# STEP 10: Test the Fine-tuned Model
# =============================================================================

finetuned_dir = "./llama1b-medical-soap-lora"   # or "./llama1b-medical-soap-merged" if you merged
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Reloading fine-tuned model for inference in FP32...")
tokenizer = AutoTokenizer.from_pretrained(finetuned_dir)
model = AutoModelForCausalLM.from_pretrained(
    finetuned_dir,
    device_map="auto",
    torch_dtype=torch.float32,   # force fp32 to avoid mismatch errors!
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device)
model.eval()

def test_model(prompt, max_length=512):
    test_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a medical assistant. Given a medical dialogue between a patient and healthcare provider, generate a SOAP (Subjective, Objective, Assessment, Plan) summary.

<|eot_id|><|start_header_id|>user<|end_header_id|>

Medical Dialogue:
{prompt}

Please provide a SOAP summary for this medical dialogue.

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    inputs = tokenizer(test_prompt, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(test_prompt):]

# Example test
sample_dialogue = """
Patient: Hi doctor, I've been having chest pain for the past two days.
Doctor: Can you describe the pain? Is it sharp or dull?
Patient: It's a sharp pain that comes and goes, especially when I take deep breaths.
Doctor: Any shortness of breath or sweating?
Patient: Yes, I've been a bit short of breath, but no sweating.
Doctor: Let me examine you. *examines patient* Your heart rate is elevated at 110 bpm, blood pressure is 140/90.
"""

print("Testing the fine-tuned model:")
response = test_model(sample_dialogue)
print(response)


# =============================================================================
# STEP 11: Save for Ollama Integration
# =============================================================================

print("Preparing model for Ollama...")
model.save_pretrained("./llama1b-medical-soap-lora")

from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
merged_model = PeftModel.from_pretrained(base_model, "./llama1b-medical-soap-lora")
merged_model = merged_model.merge_and_unload()
merged_model.save_pretrained("./llama1b-medical-soap-merged")
tokenizer.save_pretrained("./llama1b-medical-soap-merged")


# =============================================================================
# STEP 12: Download Models for Local Use
# =============================================================================

!tar -czf llama1b-medical-soap-lora.tar.gz llama1b-medical-soap-lora/
!tar -czf llama1b-medical-soap-merged.tar.gz llama1b-medical-soap-merged/

from google.colab import files
files.download('llama1b-medical-soap-merged.tar.gz')

print("Setup complete! Your fine-tuned model is ready for Ollama integration.")

CUDA available: True
GPU count: 1
GPU name: NVIDIA A100-SXM4-40GB


Loading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

MTS-Dialog-TrainingSet%20%28SDHP%29.csv: 0.00B [00:00, ?B/s]

...

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
500,1.800100,1.953554,1.819236,329918.000000,0.638809


Saving model...
Reloading fine-tuned model for inference in FP32...
Testing the fine-tuned model:

Preparing model for Ollama...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Setup complete! Your fine-tuned model is ready for Ollama integration.
